# MoCap Rasp Arena

This notebook is an implementation of the MoCap Rasp Optical Tracking Arena.

---

In [1]:
# Importing modules...
import numpy as np
import socket
import time
import cv2

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.camera import Camera
from modules.vision.synchronizer import Synchronizer
from modules.vision.multiple_view import collinear_order

from modules.integration.client import Client
from modules.integration.mocaprasp.server import MoCapRasp_Server

from intrinsic import * # Intrinsic Calibration 

# Setting Up

All `Camera` and `Client` objects will be instanciated right along with the `Server` object.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Create clients
for K, D in zip(intrinsic_matrices, distortion_coefficients):
    # Generate associated camera model
    camera = (Camera(# Intrinsic Parameters
                     resolution=(960, 720),
                     intrinsic_matrix=K.copy(),
 
                     # Fisheye Lens Distortion Model
                     distortion_model='fisheye',
                     distortion_coefficients=D.copy()
                     ))
    
    clients.append(Client(camera=camera))

# Create server
server = MoCapRasp_Server(clients=clients,
                          server_address=('0.0.0.0', 8888))

[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


# Extrinsic Calibration

The Cameras' Extrinsic Parameters will be estimated. For that, a calibration routine will be requested and the data will be post-processed for the parameter estimation.

---

In [ ]:
# Register clients
server.register_clients()

In [3]:
# Capture specifications
blob_count = 3 # Number of expected markers
delay_time = 10.0 # In seconds
capture_time = 60.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 40 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

In [4]:
verbose = True

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = delay_time + 5.0 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Receiving messages
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)
        IP, _ = address # FIX THIS

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_ips[IP] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

[SERVER] Waiting delay...
[SERVER] Timeout set to 15.0 seconds

> Received message from Client 2 (10.0.0.103, 60428)
> Received message from Client 3 (10.0.0.105, 33779)
> Received message from Client 0 (10.0.0.104, 41039)
> Received message from Client 1 (10.0.0.102, 52263)
> Received message from Client 2 (10.0.0.103, 60428)
> Received message from Client 3 (10.0.0.105, 33779)
> Received message from Client 1 (10.0.0.102, 52263)
> Received message from Client 0 (10.0.0.104, 41039)
> Received message from Client 2 (10.0.0.103, 60428)
> Received message from Client 3 (10.0.0.105, 33779)
> Received message from Client 0 (10.0.0.104, 41039)
> Received message from Client 1 (10.0.0.102, 52263)
> Received message from Client 2 (10.0.0.103, 60428)
> Received message from Client 3 (10.0.0.105, 33779)
> Received message from Client 1 (10.0.0.102, 52263)
> Received message from Client 0 (10.0.0.104, 41039)
> Received message from Client 2 (10.0.0.103, 60428)
> Received message from Client 3 (1

In [6]:
# Post-processing
for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 
        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

	No blobs were detected - 0.000 s
	Wrong blob count or corrupted message
Corrupted Message: [4.3750000e+02 1.3700000e+02 3.6055512e+00 4.6750000e+02 1.3400000e+02
 3.6055512e+00 2.5013000e-02]
	Wrong blob count or corrupted message
Corrupted Message: [4.3659421e+02 1.3672464e+02 3.7962162e+00 4.6650000e+02 1.3300000e+02
 3.6055512e+00 5.0011002e-02]
	Wrong blob count or corrupted message
Corrupted Message: [4.3600000e+02 1.3533333e+02 3.9712381e+00 4.6562744e+02 1.3221568e+02
 3.6131675e+00 7.4998997e-02]
	Wrong blob count or corrupted message
Corrupted Message: [4.3540579e+02 1.3427536e+02 3.7962162e+00 4.6500000e+02 1.3100000e+02
 2.8284271e+00 9.9992000e-02]
	Wrong blob count or corrupted message
Corrupted Message: [4.3462744e+02 1.3321568e+02 3.6131675e+00 4.6430304e+02 1.3030302e+02
 2.9554389e+00 1.2498300e-01]
	Wrong blob count or corrupted message
Corrupted Message: [4.3400000e+02 1.3233333e+02 3.9712381e+00 4.6400000e+02 1.3000000e+02
 2.8284271e+00 1.4996900e-01]
	Wrong blob 

In [7]:
# Playback a camera image feed in fidelity time
playback = True

if playback:
    ID = 0 # Camera ID to be watched
        
    # Converting to np arrays
    sync_PTS = np.array(server.clients[ID].synchronizer.sync_PTS)
    blob_data = np.array(server.clients[ID].synchronizer.sync_blobs)
    
    # Generating frames
    images = []
    for PTS, blobs in zip(sync_PTS, blob_data):
        image = np.zeros(server.clients[ID].camera.image_shape)
        image = cv2.putText(image, str(PTS), 
                                    [40, 40], 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, 
                                    (255, 0, 0), 
                                    1, 
                                    cv2.LINE_AA)

        all_ordered_blobs_per_frame = collinear_order(blobs, (1.0, 10.30e-2 / 5.35e-2))

        if all_ordered_blobs_per_frame is not None:
            for tag, blob in zip(['A', 'B', 'C'], all_ordered_blobs_per_frame):

                image = cv2.putText(image, tag, 
                                    blob.astype(int), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.5, 
                                    (255, 255, 255), 
                                    1, 
                                    cv2.LINE_AA)
            
        images.append(image)

    # Getting delay between each frame
    delays = sync_PTS[1:] - sync_PTS[:-1]

    # Playing the animation
    for delay, image in zip(delays, images):
        cv2.imshow(f'Camera {ID} Feed', image)
        cv2.waitKey(int(1e3 * delay))

    # Closing all open windows 
    cv2.destroyAllWindows()

In [ ]:
# Calibrate multiple view
wand_distances = np.array([5.35e-2, 10.30e-2, 15.70e-2]) # In meters

wand_blobs = [client.synchronizer.sync_blobs for client in server.clients]

if not server.multiple_view.calibrate(wand_blobs, wand_distances, verbose):
    sys.exit() # Calibration failed! 

In [9]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

In [10]:
# Perform bundle adjustment
n_observations = 72
server.multiple_view.bundle_adjustment(wand_blobs, wand_distances, n_observations)

In [11]:
# Create the Scene Viewer
scene = Viewer3D(title='Adjusted Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Reference Update

Right after the Extrinsic Calibration, a new reference will be set to be the new scene's canonical frame. For that, a new reference will be requested.

---

In [12]:
# Register clients
server.register_clients()

''

In [31]:
# Capture specifications
blob_count = 3 # Number of expected markers
delay_time = 1.0 # In seconds
capture_time = 5.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 40 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


In [32]:
verbose = True

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = delay_time + 5.0 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Receiving messages
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)
        IP, _ = address # FIX THIS

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_ips[IP] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

[SERVER] Waiting delay...
[SERVER] Timeout set to 10.0 seconds

> Received message from Client 2 (10.0.0.103, 39143)
> Received message from Client 2 (10.0.0.103, 50438)
> Received message from Client 0 (10.0.0.104, 45055)
> Received message from Client 0 (10.0.0.104, 40494)
> Received message from Client 1 (10.0.0.102, 48107)
> Received message from Client 1 (10.0.0.102, 59123)
> Received message from Client 2 (10.0.0.103, 39143)
> Received message from Client 3 (10.0.0.105, 43450)
> Received message from Client 0 (10.0.0.104, 45055)
> Received message from Client 3 (10.0.0.105, 51211)
> Received message from Client 1 (10.0.0.102, 48107)
> Received message from Client 2 (10.0.0.103, 39143)
> Received message from Client 3 (10.0.0.105, 43450)
> Received message from Client 0 (10.0.0.104, 45055)
> Received message from Client 2 (10.0.0.103, 39143)
> Received message from Client 1 (10.0.0.102, 48107)
> Received message from Client 3 (10.0.0.105, 43450)
> Received message from Client 0 (1

In [33]:
# Post-processing
for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 
        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = blob_data[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(blob_data).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

	Detected Blobs - 0.000 s
	[[436.5       427.5         3.1622777]
	 [456.5       413.5         3.1622777]
	 [442.5       406.          3.6055512]]
	Data Accepted!
	Detected Blobs - 0.000 s
	[[436.5       427.5         3.1622777]
	 [456.5       413.5         3.1622777]
	 [442.5       406.          3.6055512]]
	Data Refused!
	Detected Blobs - 0.025 s
	[[436.75555   427.5         4.60857  ]
	 [456.375     413.53125     4.8545074]
	 [442.34344   406.1111      4.692134 ]]
	Data Accepted!
	Detected Blobs - 0.050 s
	[[436.625     427.53125     4.8545074]
	 [456.375     413.53125     4.8545074]
	 [442.34344   406.1111      4.692134 ]]
	Data Accepted!
	Detected Blobs - 0.075 s
	[[436.5       427.5         5.0990195]
	 [456.375     413.53125     4.8545074]
	 [442.34344   406.1111      4.692134 ]]
	Data Accepted!
	Detected Blobs - 0.100 s
	[[436.625     427.53125     4.8545074]
	 [456.53125   413.625       4.8545074]
	 [442.34344   406.1111      4.692134 ]]
	Data Accepted!
	Detected Blobs - 0.125

In [34]:
# Measured distances between perpendicularly matched marker distances
# Distances: [D_x, D_y]
wand_distances = np.array([15.05e-2, 10.10e-2]) # In meters

# Triangulation pair
pair = (0, 2) # Diagonal pairs seems to produce more stable results

wand_blobs = [server.clients[ID].synchronizer.async_blobs for ID in pair]

# Update reference
server.multiple_view.update_reference(wand_blobs, wand_distances, pair)

In [35]:
# Create the Scene Viewer
scene = Viewer3D(title='Updated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add new reference
scene.add_frame(np.eye(4), 'Reference', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Standard Capture

With all calibration done, a standard capture routine can be requested for the arena's usual operation.

---

In [70]:
# Register clients
server.register_clients()

[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


In [71]:
# Capture specifications
blob_count = 1 # Number of expected markers
delay_time = 1.0 # In seconds
capture_time = 20.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 40 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(delay_time, synchronizer):
    sys.exit() # Capture request failed!

In [72]:
verbose = True

print('[SERVER] Waiting delay...')
time.sleep(delay_time)

timeout = delay_time + 5.0 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

triangulation = 0
sync_triangulated_markers = np.empty((server.clients[0].synchronizer.sync_PTS.size, blob_count, 3))
all_triangulated_markers = []

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)
        IP, _ = address # FIX THIS

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_ips[IP] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float32)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print('\tEmpty message')

        continue # Jump to wait for the next message

    # Extracting the message's PTS
    PTS = message[-1] # Last element of the message 

    # Valid message is [u, v, A] per blob and the PTS of the message
    if message.size !=  3 * blob_count + 1:

        if message.size == 1: # Only PTS
            if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

        else: 
            if verbose: 
                print(f'\tWrong blob count or corrupted message')
                print(f'Corrupted Message: {message}')

        continue # Jump to wait for the next message

    # Extracting blob data (coordinates & area)
    blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

    # Extracting centroids
    blob_centroids = blob_data[:,:2] # Ignoring their area

    # Undistorting blobs centroids
    undistorted_blobs = server.clients[ID].camera.undistort_points(blob_centroids)          

    # Print blobs
    if verbose:
        print(f'\tDetected Blobs - {PTS :.3f} s')
        print('\t' + str(blob_data).replace('\n', '\n\t'))

    # Save data
    valid_data = server.clients[ID].synchronizer.add_data(undistorted_blobs, PTS)

    if verbose: 
        if valid_data:
            print('\tData Accepted!')
        else:
            print('\tData Refused!')

    # Check for available interpolated data
    available = []
    synchronizers = [c.synchronizer for c in server.clients]
    for ID, S in enumerate(synchronizers):
        # Is there interpolated data? Non-interpolated blobs are negative!
        if np.any(S.sync_blobs[triangulation] >= 0): 
            available.append(ID)

    # If at least two cameras have interpolated data, triangulate 
    if len(available) >= 2:
        # Get the first two available
        pair = available[:2]

        blob_pair = [synchronizers[pair[0]].sync_blobs[triangulation],
                     synchronizers[pair[1]].sync_blobs[triangulation]]
        
        triangulated_markers = server.multiple_view.triangulate_by_pair(pair, blob_pair)
        all_triangulated_markers.append(triangulated_markers)

        # Update last triangulation flag
        if triangulation < sync_triangulated_markers.shape[0] - 1:
            triangulation += 1

all_triangulated_markers = np.hstack(all_triangulated_markers)

[SERVER] Waiting delay...
[SERVER] Timeout set to 6.0 seconds

> Received message from Client 2 (10.0.0.103, 35687)
	No blobs were detected - 0.000 s
> Received message from Client 3 (10.0.0.105, 53260)
	Detected Blobs - 0.000 s
	[[501.5       359.5         3.1622777]]
	Data Accepted!
> Received message from Client 0 (10.0.0.104, 43058)
	Detected Blobs - 0.000 s
	[[441.5       371.5         3.1622777]]
	Data Accepted!
> Received message from Client 1 (10.0.0.102, 49323)
	Detected Blobs - 0.000 s
	[[562.9375    350.06253     4.3033423]]
	Data Accepted!
> Received message from Client 2 (10.0.0.103, 35687)
	No blobs were detected - 0.025 s
> Received message from Client 3 (10.0.0.105, 53260)
	Detected Blobs - 0.025 s
	[[501.93747   359.93747     4.3033423]]
	Data Accepted!
> Received message from Client 1 (10.0.0.102, 49323)
	Detected Blobs - 0.025 s
	[[563.2962   350.42383    4.844615]]
	Data Accepted!
> Received message from Client 0 (10.0.0.104, 43058)
	Detected Blobs - 0.025 s
	[[441.

# Capture Profile

---

In [73]:
# Create the Scene Viewer
scene = Viewer3D(title='Capture Profile', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add new reference
scene.add_frame(np.eye(4), 'Reference', axis_size=0.4)

# Add triangulated markers to the scene
scene.add_points(all_triangulated_markers, f'Markers')

# Plot scene
scene.figure.show(renderer='notebook_connected')